# S1W2D3 - PyTorch 核心：`Tensor`与`Autograd`

**今日目标**

1. 熟练掌握`torch.Tensor`的基本操作，把它当作NumPy来使用
2. 理解并亲手实践PyTorch的自动求导机制（Autograd）
3. 搞清楚`requires_gard`，`.backward()`和`.gard`之间的“三角关系”。

## PyTorch 的“砖块”：`torch.Tensor` (张量)**

`Tensor` 是 PyTorch 中最基本的数据结构，它就是一个**多维数组**。
  * **它和 NumPy 的 `ndarray` 有什么关系？**

      * 它们几乎一模一样。你可以像操作 NumPy 数组一样对 `Tensor` 进行索引、切片、数学运算。
      * 它们甚至可以高效地互相转换。

  * **它和 NumPy 的 `ndarray` 有什么区别？(核心)**

    1.  **GPU 加速**： `Tensor` 可以被无缝地移动到 NVIDIA GPU 上进行计算，实现百倍千倍的加速。
    2.  **自动求导**： `Tensor` 是 PyTorch 自动求导系统 `Autograd` 的核心。

**代码实践：`Tensor` 的创建与操作**

In [14]:
import torch
import numpy as np

# --- Tensor 创建 ---
# 1. 从 Python 列表创建
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(f"t1 (from list):\n{t1}\n")

t1 (from list):
tensor([[1, 2, 3],
        [4, 5, 6]])



In [15]:
# 2. 创建全零或全一（需要指定形状）
t2 = torch.zeros((2, 3))  # 2行3列
t3 = torch.ones((2, 3)) # 2行3列
print(f"t2 (zero):\n{t2}\n")
print(f"t3 (zero):\n{t3}\n")

t2 (zero):
tensor([[0., 0., 0.],
        [0., 0., 0.]])

t3 (zero):
tensor([[1., 1., 1.],
        [1., 1., 1.]])



In [16]:
# 3. 创建随机数（非常实用）
t4 = torch.randn((2,3))   # 随机数以0为均值，满足正态分布
print(f"t3 (randn):\n{t4}\n")

t3 (randn):
tensor([[-0.7883, -0.7182, -0.6684],
        [-1.4609,  0.5208,  1.6947]])



In [17]:
# 4. 和 NumPy 互相转换
np_array = np.array([1, 2, 3])
t5 = torch.from_numpy(np_array) # numpy -> tensor
print(f"t5 (from numpy):\n{t5}\n")

np_array_back = t5.numpy()      # tensor -> numpy
print(f"Back to numpy:\n{np_array_back}\n")

t5 (from numpy):
tensor([1, 2, 3])

Back to numpy:
[1 2 3]



In [18]:
# --- Tensor 属性与操作 ---
print(f"t1 shape: {t1.shape}")      # 形状
print(f"t1 dtype: {t1.dtype}")      # 数据类型 (默认 torch.int64)
print(f"t4 dtype: {t4.dtype}")      # (默认 torch.float32)

# 索引 (和NumPy一样)
print(f"t1的第一行: {t1[0]}")
print(f"t1的[0, 2]元素: {t1[0, 2]}\n")

# 数学运算 (和NumPy一样)
t_a = torch.tensor([[1, 1], [2, 2]])
t_b = torch.tensor([[3, 3], [4, 4]])
print(f"Tensor相加:\n{t_a + t_b}\n")
print(f"Tensor逐元素相乘:\n{t_a * t_b}\n")

t1 shape: torch.Size([2, 3])
t1 dtype: torch.int64
t4 dtype: torch.float32
t1的第一行: tensor([1, 2, 3])
t1的[0, 2]元素: 3

Tensor相加:
tensor([[4, 4],
        [6, 6]])

Tensor逐元素相乘:
tensor([[3, 3],
        [8, 8]])



## 2. 通往 GPU 的“桥梁”：`.to('cuda')`

这是 PyTorch 相比 NumPy 的**巨大优势**。要让计算在你的 RTX 3050 上飞起来，你只需要做一件事：把你的数据和模型都“搬”到 GPU 上。

**代码实践 2：检查并使用 GPU**

In [19]:
import torch

# 1. 检查 CUDA (NVIDIA GPU) 是否可用
# 这是 PyTorch 代码的“标准开头”
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"我们将要使用的设备是: {device}")

# 2. 创建一个 Tensor，默认在 CPU 上
t_cpu = torch.randn((2, 2))
print(f"t_cpu 所在的设备: {t_cpu.device}")

# 3. 将 Tensor 移动到 GPU 上
if device == 'cuda':
    t_gpu = t_cpu.to(device) # 或者 .to('cuda')
    print(f"t_gpu 所在的设备: {t_gpu.device}")
    
    # 警告：CPU 上的 Tensor 不能和 GPU 上的 Tensor 直接计算
    # 下面这行会报错:
    # t_cpu + t_gpu 
    
    # 必须在同一设备上
    t_gpu_2 = torch.randn((2, 2)).to(device)
    result_gpu = t_gpu + t_gpu_2
    print(f"GPU 上的计算结果:\n{result_gpu}")
    
    # 把结果从 GPU 搬回 CPU (比如为了用 numpy 或 print)
    result_cpu = result_gpu.cpu()
    print(f"搬回 CPU 后的结果:\n{result_cpu}")

我们将要使用的设备是: cuda
t_cpu 所在的设备: cpu
t_gpu 所在的设备: cuda:0
GPU 上的计算结果:
tensor([[-1.6437, -0.2288],
        [ 0.2626, -0.8079]], device='cuda:0')
搬回 CPU 后的结果:
tensor([[-1.6437, -0.2288],
        [ 0.2626, -0.8079]])


## 3. PyTorch 的“魔法”：`Autograd` 自动求导

这是今天的**核心**。`Autograd` 就是 PyTorch 对“反向传播”的工程实现。你再也不用手动算梯度了，PyTorch 会帮你搞定一切。

**它是如何工作的？**
当你告诉 PyTorch 某个 `Tensor` **需要梯度** 时（通过设置 `requires_grad=True`），PyTorch 就会在后台默默地为你构建一个**计算图 (Computation Graph)**。这个图记录了所有与该 `Tensor` 相关的计算。

当你对最终的计算结果（比如 `loss`）调用 `.backward()` 时，PyTorch 就会沿着这个图反向传播，自动计算出图中所有 `requires_grad=True` 的 `Tensor` 的梯度，并把它们存放在各自的 `.grad` 属性中。

**三个关键点：**

1.  **`requires_grad=True`**: 像一个“开关”，告诉 `Autograd`：“请开始追踪我对这个 `Tensor` 的所有操作”。（模型参数默认是 `True`，输入数据默认是 `False`）。
2.  **`loss.backward()`**: 像一个“扳机”，触发反向传播。PyTorch 会自动计算所有被追踪的 `Tensor` 的梯度。
3.  **`tensor.grad`**: 像一个“信箱”，用来存放 `backward()` 计算出的梯度值。

**代码实践 3：亲手实践自动求导**

我们将手动复现昨天的理论：计算函数 $y = x^2 + 2x + 1$ 在 $x=2$ 时的导数 $\frac{dy}{dx}$。
（我们手动计算：$\frac{dy}{dx} = 2x + 2$，当 $x=2$ 时，导数应为 $2 \times 2 + 2 = 6$）

In [23]:
import torch

# 1. 创建 x，并打开“追踪”开关
x = torch.tensor(2.0, requires_grad=True)
print(f"x: {x}\n")

# 2. 定义函数 y
# PyTorch 会自动记录这个计算过程
y = x**2 + 2*x + 1
print(f"y (计算结果): {y}\n")

# 3. 触发反向传播
# PyTorch 会自动计算 dy/dx
y.backward()

# 4. 查看 x 的梯度 (结果存放在 .grad 属性中)
# 结果应该是 6
print(f"x 的梯度 (dy/dx): {x.grad}\n") 


# --- 代码实践 4：(重要!) 梯度的累加特性 ---
print("--- 实践 4: 梯度累加 ---")
# 假设我们又进行了一次计算 (比如在训练的下一个step)
# 注意：x.grad 里现在的值是 6
y2 = x**2 + x 
# (y2 的导数是 2x + 1，在 x=2 时，导数是 5)
# x.grad.zero_()
y2.backward()

# 再次查看 x.grad
# 你觉得会是 5 吗？
print(f"第二次 backward 后 x 的梯度: {x.grad}\n")
# 结果会是 11 (即 6 + 5)

# **结论 (面试高频):**
# PyTorch 的梯度在 .grad 中是默认 **累加 (accumulate)** 的。
# 这就是为什么在每个训练循环 (training loop) 的开始，
# 我们必须显式地调用 `optimizer.zero_grad()` 来清空上一轮的梯度！

# 如何手动清零？
x.grad.zero_() # 使用 .zero_() (带下划线表示 in-place 原地操作)
print(f"清零后 x 的梯度: {x.grad}")

x: 2.0

y (计算结果): 9.0

x 的梯度 (dy/dx): 6.0

--- 实践 4: 梯度累加 ---
第二次 backward 后 x 的梯度: 11.0

清零后 x 的梯度: 0.0


## W2D3 今日行动清单

1.  **✅ 运行代码：** 亲手将上面 4 个实践代码块在你的环境中（确保 PyTorch 和 CUDA 已装好）运行一遍。
2.  **✅ 理解 GPU：** 确保你理解 `t.to(device)` 的作用，以及为什么 CPU 和 GPU 的张量不能直接通信。
3.  **✅ 掌握 Autograd：** **这是今天的核心！** 确保你亲眼看到了 `x.grad` 是如何从 `None` 变为 `6.0`，再变为 `11.0`，最后又被清零的。这个小实验是理解整个 PyTorch 训练循环的关键。
4.  **✅ 思考题 (面试模拟)：**
      * “在 PyTorch 的训练循环中，为什么 `optimizer.zero_grad()` 这一行是必需的？它通常被放在哪里？”